## Price movement from 1:45PM to 2:30PM

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
# Ignore all candlesticks at 14:30 & 14:45
data = dataset[(dataset.index.hour != 14) | ( (dataset.index.hour == 14) & (dataset.index.minute != 30) & (dataset.index.minute != 45))]
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2024-07-04 14:05:00,1308.9,1309.5,1307.3,1308.3,3619
2024-07-04 14:10:00,1308.2,1308.2,1306.4,1306.4,4173
2024-07-04 14:15:00,1306.6,1307.6,1304.3,1304.5,10563


In [7]:
def cal_first_close(tick):
  if not tick.empty:
    return tick[0]


def cal_high_before(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 910]
  tick = tick[100*tick.index.hour+tick.index.minute < 1340]
  return tick.max()

def cal_high_after(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 1340]
  tick = tick[100*tick.index.hour+tick.index.minute < 1430]
  return tick.max()


def cal_low_before(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 910]
  tick = tick[100*tick.index.hour+tick.index.minute < 1340]
  return tick.min()


def cal_low_after(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 1340]
  tick = tick[100*tick.index.hour+tick.index.minute < 1430]
  return tick.min()


def cal_price(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1340]
  if not tick.empty:
    return tick[0]

def cal_close(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1425]
  if not tick.empty:
    return tick[0]

In [8]:
data['first_close'] = data.Close
data['price'] = data.Close
data['prev_high'] = data.High
data['prev_low'] = data.Low
data['next_high'] = data.High
data['next_low'] = data.Low
price = data.resample("D").agg({
    'first_close': cal_first_close,
    'prev_high':cal_high_before,
    'prev_low': cal_low_before,
    'next_high':cal_high_after,
    'next_low': cal_low_after,
    'price': cal_price,
    'Close': cal_close
    })
price = price.dropna()

In [9]:
price

,first_close,prev_high,prev_low,next_high,next_low,price,Close
Date,,,,,,,
2018-08-13,943.1,948.9,942.3,954.6,947.4,948.8,954.1
2018-08-14,954.9,957.9,952.1,961.8,956.5,958.3,960.1
2018-08-15,958.7,962.3,954.7,959.9,949.9,957.5,951.2
2018-08-16,942.0,943.5,935.5,947.3,940.2,942.0,944.5
2018-08-17,953.0,954.6,948.5,950.9,945.0,951.0,947.0
...,...,...,...,...,...,...,...
2024-06-28,1292.0,1292.4,1283.2,1288.1,1272.5,1283.0,1277.9
2024-07-01,1277.5,1278.2,1271.7,1285.8,1276.3,1276.1,1284.8
2024-07-02,1286.6,1297.5,1284.6,1298.8,1293.7,1298.3,1296.6


In [10]:
price[(price.next_high - price.price > 3) & (price.price - price.next_low > 3)]

,first_close,prev_high,prev_low,next_high,next_low,price,Close
Date,,,,,,,
2018-10-11,945.9,944.5,923.3,935.3,921.5,929.0,922.1
2018-10-15,940.2,939.3,924.1,931.0,922.2,926.5,922.5
2018-10-23,914.1,914.5,891.1,904.4,891.2,894.5,896.8
2018-10-25,860.0,884.6,860.7,887.0,877.0,883.7,887.0
2018-11-06,905.5,906.3,899.1,904.3,894.5,900.9,895.6
...,...,...,...,...,...,...,...
2024-06-18,1311.9,1318.7,1311.4,1322.4,1309.2,1316.2,1309.5
2024-06-20,1317.5,1327.8,1318.1,1324.3,1316.7,1321.0,1323.6
2024-06-21,1321.2,1321.7,1312.1,1323.4,1311.7,1318.8,1317.5


In [14]:
price['percent'] = 100 * (price.price - price.Close.shift(1)) / price.Close.shift(1)
price['returns'] = (price.price - price.first_close) / (price.prev_high - price.prev_low)
price['return'] = 1000 * (price.Close - price.price) / price.price

In [17]:
price[(price['return'] < -25) | (price['return'] > 25)]

,first_close,prev_high,prev_low,next_high,next_low,price,Close,percent,returns,return
Date,,,,,,,,,,
2020-03-17,682.0,696.1,675.3,704.4,679.0,681.3,704.0,1.233284,-0.033654,33.318656
2020-05-08,747.0,792.4,748.7,784.8,730.0,785.0,739.7,5.567509,0.869565,-57.707006
2020-06-11,835.3,838.5,821.4,822.9,798.6,823.0,799.3,-1.977132,-0.719298,-28.797084
2022-04-25,1442.7,1445.4,1386.4,1394.0,1346.0,1392.0,1349.6,-3.601108,-0.859322,-30.459770
2022-04-27,1377.3,1380.3,1359.2,1405.5,1365.0,1365.0,1399.9,-1.791496,-0.582938,25.567766
2022-11-10,953.5,959.3,936.0,941.0,906.2,941.1,913.0,-2.203055,-0.532189,-29.858676
2022-12-02,1027.4,1054.7,1025.2,1097.4,1053.0,1053.5,1097.4,1.503035,0.884746,41.670622
2022-12-06,1094.3,1096.3,1074.0,1085.1,1051.2,1082.5,1051.2,-1.258780,-0.529148,-28.914550
2023-02-01,1128.7,1127.0,1111.7,1125.6,1085.6,1120.6,1089.1,-0.479574,-0.529412,-28.109941
